In [61]:
import glob
import cv2
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [49]:
frontPath = glob.glob("D:\\Dataset\\De_id_sample_data\\Sample_data2\\front\\*.jpg")
smilePath = glob.glob("D:\\Dataset\\De_id_sample_data\\Sample_data2\\smile\\*.jpg")
fullPath = frontPath + smilePath

In [20]:
# # img size = (518*2,345*2)
# imgSize = (518*2,345*2)
# a =cv2.imread(frontPath[0])
# a= cv2.resize(a,imgSize)
# cv2.imshow("a",a)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [48]:
def generalization(img):
    imgSize = (518*2,345*2)
    img = cv2.resize(img, imgSize)
    img = img/255.0
    img = np.expand_dims(img,axis=0)
    return img

In [53]:
for i in range(len(fullPath)):
    print(i,end="\t")
    img = cv2.imread(fullPath[i])
    img = generalization(img)
    if i == 0 :
        imgSet = img
    else:
        imgSet = np.concatenate((imgSet,img),axis =0 )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74


In [54]:
imgSet.shape

(75, 690, 1036, 3)

In [108]:
Loc = glob.glob("./*.csv")
frontLoc = pd.read_csv(Loc[0])
smileLoc = pd.read_csv(Loc[1])


In [109]:
objs = [frontLoc,smileLoc]
fullLoc = pd.concat(objs,ignore_index=True)

In [110]:
del fullLoc['Unnamed: 0']
del fullLoc['paths']
del fullLoc['fileName']

In [111]:
print(fullLoc['Right X'].mean())
print(fullLoc['Right Y'].mean())
print(fullLoc['Left X'].mean())
print(fullLoc['Left Y'].mean())

print(fullLoc['Right X'].std())
print(fullLoc['Right Y'].std())
print(fullLoc['Left X'].std())
print(fullLoc['Left Y'].std())

563.1066666666667
300.1333333333333
451.06666666666666
299.93333333333334
6.907835999659965
21.01886754306218
8.355825501996785
20.1891952193148


In [88]:
def normalize(df):
    df_mean=[]
    df_std=[]
    keys = df.keys()
    for i in range(len(keys)):        
        df_mean.append(df[keys[i]].mean())
        df_std.append(df[keys[i]].std())
        df[keys[i]] = df[keys[i]] - df[keys[i]].mean()
        df[keys[i]] = df[keys[i]] / df[keys[i]].std()
        
    return df, df_mean, df_std

In [112]:
normLoc = normalize(fullLoc)

In [115]:
normLoc[1]

[451.06666666666666, 299.93333333333334, 563.1066666666667, 300.1333333333333]

In [117]:
normLoc[2]

[8.355825501996785, 20.1891952193148, 6.907835999659965, 21.01886754306218]

In [118]:
normLoc[0]

,Left X,Left Y,Right X,Right Y
0,-0.486686,0.102365,-1.173546,0.088809
1,-0.007978,-0.392949,1.576953,-0.529683
2,0.111699,0.052834,-0.449731,0.041233
3,-1.085071,-1.334047,-1.897362,-1.243327
4,-0.726040,-0.739670,-1.028783,-0.719988
...,...,...,...,...
70,0.590406,1.489245,0.418848,1.516098
71,0.231376,-0.888264,3.024584,-0.577259
72,-0.127655,-0.293887,-0.160205,-0.196649
73,1.308468,-0.937795,0.129322,-0.910293


In [119]:
trainImg, testImg = imgSet[:70],imgSet[70:]

In [170]:
dfTrainLabel, dfTestLabel = normLoc[0].loc[:69],normLoc[0].loc[70:]
trainLabel = dfTrainLabel.to_numpy()
testLabel = dfTestLabel.to_numpy()

In [137]:
trainImg.shape

(70, 690, 1036, 3)

In [142]:
testImg.shape

(5, 690, 1036, 3)

In [148]:
trainLabel.shape

(70, 4)

In [151]:
testLabel.shape

(5, 4)

In [172]:
inputs = keras.Input(shape=(690,1036,3))
conv1 = keras.layers.Conv2D(16,(3,3),activation='relu')(inputs)
maxp1 = keras.layers.MaxPool2D((2,2))(conv1)
conv2 = keras.layers.Conv2D(32,(3,3),activation='relu')(maxp1)
maxp2 = keras.layers.MaxPool2D((2,2))(conv2)
conv3 = keras.layers.Conv2D(64,(3,3),activation='relu')(maxp2)
maxp3 = keras.layers.MaxPool2D((2,2))(conv3)
conv4 = keras.layers.Conv2D(64,(3,3),activation='relu')(maxp3)
maxp4 = keras.layers.MaxPool2D((2,2))(conv4)

flat = keras.layers.Flatten()(maxp4)
dense64 = keras.layers.Dense(64,activation = 'relu')(flat)
dense32 = keras.layers.Dense(32,activation = 'relu')(dense64)
outputs = keras.layers.Dense(4)(dense32)

model= keras.Model(inputs=inputs,outputs= outputs)

model.compile(loss = 'mae',metrics=['acc'],optimizer="rmsprop")

In [174]:
hist = model.fit(trainImg,trainLabel,epochs=20,batch_size=2,validation_data=(testImg,testLabel))

Train on 70 samples, validate on 5 samples
Epoch 1/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.3302 - acc: 0.6857 - val_loss: 0.4765 - val_acc: 0.8000
Epoch 2/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.3008 - acc: 0.7571 - val_loss: 0.5333 - val_acc: 0.8000
Epoch 3/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.3193 - acc: 0.7571 - val_loss: 0.5191 - val_acc: 0.8000
Epoch 4/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.2687 - acc: 0.7571 - val_loss: 0.5420 - val_acc: 0.8000
Epoch 5/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.2898 - acc: 0.7286 - val_loss: 0.5367 - val_acc: 0.8000
Epoch 6/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.2641 - acc: 0.8000 - val_loss: 0.4618 - val_acc: 0.8000
Epoch 7/20
70/70 [==============================] - 2s 32ms/sample - loss: 0.2130 - acc: 0.7429 - val_loss: 0.4708 - val_acc: 0.8000
Epoch 8/20
70/70 [========

In [194]:
a = np.expand_dims(testImg[0],axis =0)
b = model.predict(a)

In [195]:
b

array([[ 0.01264799,  1.1502572 , -0.22871496,  1.2674732 ]],
      dtype=float32)

In [ ]:
plt.imshow(testImg[0])
# * std
for i in range(len(normLoc[2])):
    b[0][i] = b[0][i] * normLoc[2][i]
# + mean
for i in range(len(normLoc[1])):
    b[0][i] = b[0][i] + normLoc[1][i]

In [208]:
b[0]

array([451.17236, 323.1561 , 561.52673, 326.7742 ], dtype=float32)

In [205]:
# cv2.circle(testImg[0],(b[0][0],b[0][1]),2,(0,0,0),-1)
cv2.imshow('a',testImg[0])
cv2.waitKey()
cv2.destroyAllWindows()

In [211]:
test = testImg[0]
cv2.circle(test,(b[0][2],b[0][3]),10,(255,0,0),-1)
cv2.imshow('a',test)
cv2.waitKey()
cv2.destroyAllWindows()